In [1]:
# this cell is a placeholder for the jupyter notebook
galaxy='NGC5005'

# Writing the chosen galaxy name in a text file
f = open("python/dontlook.txt", "w")
f.write(galaxy)
f.close()
# apparently you need to open the file again and read it and close it for the changes to stick
f = open("python/dontlook.txt", "r")
f.close()

data/NGC5533/ngc5005_disk.txt


In [ ]:
# this cell is a placeholder for the .py file

# Read the file where the chosen galaxy name is located
f = open('python/dontlook.txt', 'r')
f = f.read()

##############################
### Import data/text files ###
##############################
if galaxy = 'NGC5005':
    dataFile = 'ngc5005_data.txt'
    diskFile='ngc5005_disk.txt'
    bulgeFile = 'ngc5005_bulge.txt'
    gasFile='ngc5005_gas.txt'
    
    gpref=1
    dpref=1
    bpref=1
    rho00=1
    rc=1
elif galaxy='NGC7814':
    dataPath = strs('data/'+galaxy )" 'NGC5005/ngc5005_data.txt'
    diskPath = 'data/NGC5005/ngc5005_disk.txt'
    bulgePath = 'data/NGC5005/ngc5005_bulge.txt'
    gasPath='data/NGC5005/ngc5005_gas.txt'
    
    gpref=1
    dpref=1
    bpref=1
    rho00=1
    rc=1
elif galaxy='NGC891':
    dataPath = 'data/NGC5005/ngc5005_data.txt'
    diskPath = 'data/NGC5005/ngc5005_disk.txt'
    bulgePath = 'data/NGC5005/ngc5005_bulge.txt'
    gasPath='data/NGC5005/ngc5005_gas.txt'
    
    gpref=1
    dpref=1
    bpref=1
    rho00=1
    rc=1
elif galaxy='NGC5533':
    dataPath = 'data/NGC5005/ngc5005_data.txt'
    diskPath = 'data/NGC5005/ngc5005_disk.txt'
    bulgePath = 'data/NGC5005/ngc5005_bulge.txt'
    gasPath='data/NGC5005/ngc5005_gas.txt'
    
    gpref=1
    dpref=1
    bpref=1
    rho00=1
    rc=1
    #dataFunciton(dataPath,diskPath,bulgePath,gasPath)
    
#*******************************
#Assigning txt file info
#*******************************
dataPath=str('data/'+galaxy+'/'+dataFile)
diskPath=str('data/'+galaxy+'/'+diskFile)
bulgePath=str('data/'+galaxy+'/'+bulgeFile)
gasPath=str('data/'+galaxy+'/'+gasFile)

# Datapoints:
data = dp.getXYdata_wXYerr(dataPath)
r_dat = np.asarray(data['xx'])
v_dat = np.asarray(data['yy'])
v_err0 = np.asarray(data['ex'])
v_err1 = np.asarray(data['ey'])
# Disk:
disk_data = dp.getXYdata(diskPath)
disk_raw = np.asarray(disk_data['yy'])
# Bulge:
bulge_data = dp.getXYdata(bulgePath)
bulge_raw = np.asarray(bulge_data['yy'])
# Gas:
gas_data = dp.getXYdata(gasPath)
gas_raw = np.asarray(gas_data['yy'])  

weighdata = 1/v_err1
G = 4.30091e-6            # gravitational constant (kpc/solar mass*(km/s)^2)

def interpd(x,y):
    return InterpolatedUnivariateSpline(x,y,k=5)

def bulge(r,bpref):
    polynomial = interpd(r_dat,bpref*bulge_raw)   
    return polynomial(r)

def disk(r,dpref):
    polynomial = interpd(r_dat,dpref*disk_raw)   
    return polynomial(r)

def gas(r,gpref):
    polynomial = interpd(r_dat,gpref*gas_raw)   
    return polynomial(r)

def halo(r,rc,rho00):
    return nf.h_v(r,rc,rho0,load=False)


##################################
### Calculating total velocity ###
##################################

def totalvelocity(r,mBH,bpref,dpref,gpref):
    return np.sqrt((disk(r,dpref))**2
               +(bulge(r,bpref))**2
               +(gas(r,gpref))**2
               +(halo_BH(r,mBH))**2)

# Define components
def bulge(B):
    return B*comp7814.b

def disk(D):
    return D*comp7814.d

def halo(r,rc,rho00):
    return nf.h_v(r,rc,rho00,load=False)

def gas(G):
    return G*comp7814.g

def totalcurve(r,G,B,D,rc,rho00):
    return np.sqrt((gas(G)**2)
                   + (bulge(B)**2) 
                   + (disk(D)**2)
                   + (halo(r,rc,rho00)**2))

#best fitted prefactor values for each component, to be used as default (initial) values for widget sliders
g_dict = wi.g_fit.best_values
best_G = g_dict['G']
best_B = g_dict['B']
best_D = g_dict['D']
best_rc = g_dict['rc']
best_rho00 = g_dict['rho00']

z=comp7814.r[0]
rr=[0,z]

# Define plotting function
def f(G,B,D,rc,rho00):
    
    # Define r
    xmax=20 #kpc
    rval = np.linspace(0,11.2,19)
    
    # Plot
    plt.figure(figsize=(11,6))
    plt.xlim(0,xmax)
    plt.ylim(0,360)
    
    plt.errorbar(comp7814.r,comp7814.v_dat,yerr=comp7814.v_err1,fmt='bo',label='Data')
    
    plt.plot(comp7814.r,bulge(B),label=("Bulge"),color='orange')
    plt.plot(rr,[0,bulge(B)[0]],color='orange') #straight lining connecting halo curve to origin, for visual aesthetic
    plt.plot(comp7814.r,disk(D),label=("Disk"),color='purple')
    plt.plot(rr,[0,disk(D)[0]],color='purple') #straight lining connecting halo curve to origin, for visual aesthetic
    plt.plot(comp7814.r,halo(comp7814.r,rc,rho00),label=("Halo"),color='green')
    plt.plot(rr,[0,halo(comp7814.r,rc,rho00)[0]],color='green') #straight lining connecting halo curve to origin, for visual aesthetic
    plt.plot(comp7814.r,gas(G),label=("Gas"),color='blue')
    plt.plot(comp7814.r,totalcurve(comp7814.r,G,B,D,rc,rho00),label=("Total Curve"),color='red')
    plt.plot(rr,[0,totalcurve(comp7814.r,G,B,D,rc,rho00)[0]],color='red') #straight lining connecting halo curve to origin, for visual aesthetic

    plt.title("Interactive Rotation Curve - Galaxy: NGC7814")
  
    plt.xlabel("Radius (kpc)")
    plt.ylabel("Velocity (km/s)")
    
    
    plt.legend(loc='lower right')
    
     
    #chi squared button commented out for now bc tiny graph
    residuals = comp7814.v_dat - totalcurve(comp7814.r,G,B,D,rc,rho00)
    # Determining errors
    errors = comp7814.v_err1**2 #second term is inclination uncertainty
    # Chi squared
    chisquared = np.sum(residuals**2/errors**2)
    #chisquared = stats.chisquare(v_dat,totalcurve(r,M,bpref,dpref,rc,rho00,gpref))
    reducedchisquared = chisquared * (1/(len(comp7814.r)-6))
    
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    plt.text(10,300,r"$\chi^2$: {:.5f}".format(chisquared)+'\n'+r"Reduced: {:.5f}".format(reducedchisquared),bbox=props)
    plt.annotate('Data source: Fraternali1, Sancisi, Kamphuis. "A tale of two galaxies: light and mass in NGC 891 and NGC 7814". A&A Journal. Jun 2011',
            xy=(0, 0), xytext=(0,5),
            xycoords=('axes fraction', 'figure fraction'),
            textcoords='offset points',
            size=10, ha='left', va='bottom')
    plt.show()

# Appearance
style = {'description_width': 'initial'}
layout = {'width':'600px'}

# Define slides
G = FloatSlider(min=0, max=5, step=0.1, 
                    value=gpref,
                    description='Gas Prefactor', 
                    readout_format='.2f', 
                    orientation='horizontal', 
                    style=style, layout=layout)

B = FloatSlider(min=0, max=10, step=0.1, 
                    value=bpref, 
                    description='Bulge Prefactor', 
                    readout_format='.2f', 
                    orientation='horizontal', 
                    style=style, layout=layout)

D = FloatSlider(min=0, max=5, step=0.1, 
                    value=dpref, 
                    description='Disk Prefactor', 
                    readout_format='.2f', 
                    orientation='horizontal', 
                    style=style, layout=layout)

rc = FloatSlider(min=0, max=5, step=0.1, value=rc, description='Halo Core Radius [kpc]', readout_format='.2f', orientation='horizontal', style=style, layout=layout)
#rc = fixed(best_rc)
#rho00 = fixed(best_rho00)

rho00 = FloatSlider(min=0, max=1e9, step=1e7, 
                    value=rho00, 
                    description=r'Halo Surface Density [$M_{\odot} / pc^3$]', 
                    readout_format='.2e', 
                    orientation='horizontal', 
                    style=style, layout=layout)

# Interactive widget
def interactive_plot(f):
    interact = interactive(f,
                               G = G,
                               B = B, 
                               D = D, 
                               rc = rc,
                               rho00 = rho00,
        
                               continuous_update=False)
    return interact

# Button to revert back to Best Fit
button = Button(
    description="Best Fit",
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    icon='check')
out = Output()

def on_button_clicked(_):
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    B.value = bpref
    D.value = dpref
    rho00.value = rho00
    rc.value = rc
    G.value = gpref

button.on_click(on_button_clicked)

# displaying button and its output together
VBox([button,out,interactive_plot(f)])